<head><b><center>MACHINE LEARNING</center>

<center>Redes Neuronales Artificiales</center>
    

<center>Profesor: Gabriel Jara </center></head><br>
El presente Jupyter Notebook busca:
<ul>
    <li>Retomar conceptos sobre Redes Neuronales Artificiales (ANN).</li>
    <li>Profundizar en el mecanismo de aprendizaje de las ANN</li>
    <li>Analizar la implementación de una red neuronal sin apoyo de librerías especializadas.</li>
</ul>

Fuentes: 
<ul>
    <li><a href=https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65>Neural Network from scratch in Python</a></li>
    <li>Imágenes han sido robadas de internet y son enlaces a su correspondiente fuente. </li>
</ul> 



<b>En el capítulo anterior </b>

Presentamos las Redes Neuronales Artificiales como modelo de aprendizaje. Identificamos el <b><i>Perceptron</i></b> como unidad básica de este modelo, cuya función emula el rol de una neurona: percibe un estímulo según el cual se activa (o no) y con ello transmite una señal. 
<br> <img src=https://images.deepai.org/glossary-terms/perceptron-6168423.jpg
          width="400"/>

La naturaleza limitada de un perceptrón no permitiría que resolviera problemas muy complejos. En el contexto de Clasificación, sólo podría clasificar observaciones que fueran linealmente separables. Pero como nodos en una red organizada en capas densamente conectadas, alcanzan capacidades mucho mayores. Esta es la definición básica de una red neuronal, denominada <b><i>Multi-Layer Perceptron</i></b>. 
<br> <img src=https://miro.medium.com/v2/resize:fit:563/1*4_BDTvgB6WoYVXyxO8lDGA.png
          width="350"/>
          



          

<b>RED NEURONAL PASO A PASO</b>

Si bien regularmente vamos a aprovechar implementaciones ya disponibles, puede ocurrir que en ocasiones requiramos alguna modificación no prevista, o necesitamos controlar muy detalladamente el algoritmo que se ejecuta con la Red Neuronal. Por eso vamos a adaptar y analizar una implementación que fue publicada por Omar Aflak en https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65, la cual hemos modificado para fines de esta experiencia.          

Iremos comentando por partes la implementación. 

<b>Clase Layer</b>


In [ ]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

Esta es la clase base para las capas en la red neuronal. Tiene dos funciones principales que deben ser implementadas por las clases hijas: forward_propagation y backward_propagation. En las redes neuronales, durante la fase de entrenamiento, los datos pasan hacia adelante a través de la red (forward propagation) y luego los errores se propagan hacia atrás (backward propagation) para ajustar los pesos.

<b>Clase FCLayer</b>

In [ ]:
import numpy as np

# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        # Sólo con fines pedagógicos, incluímos un seed para controlar la aleatoriedad
        # de los pesos iniciales en la red. 
        np.random.seed(1234)
        self.weights = np.random.rand(input_size, output_size, ) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

Esta es una capa completamente conectada (Fully-Connected Layer) de la red neuronal. Cada neurona en una capa está conectada a todas las neuronas en la capa anterior y todas las neuronas en la capa siguiente.

<ul>
    <li>En el método <b>__init__</b>, se inicializan los pesos y los sesgos con valores aleatorios.</li>
    <li>El método <b>forward_propagation</b> toma la entrada y la multiplica con los pesos (y suma los sesgos) para obtener la salida. Esto se puede expresar con la ecuación: $y = W \cdot x + b$, donde $W$ son los pesos, $x$ es la entrada, $b$ es el sesgo y $y$ es la salida.</li>
    <li>El método <b>backward_propagation</b> se encarga de calcular los gradientes y actualizar los pesos y sesgos. La ecuación relevante para calcular el error de entrada es $E_{in} = E_{out} \cdot W^T$, donde $E_{out}$ es el error de salida y $W^T$ es la transposición de la matriz de pesos. Luego se actualizan los pesos y sesgos con las ecuaciones: $W = W - \alpha \cdot E_{out} \cdot X^T$, $b = b - \alpha \cdot E_{out}$, donde $\alpha$ es la <i>learning rate</i> y $X$ es la entrada.</li>
</ul>

¿Dé dónde salen las transformaciones realizadas en el paso <i>backward propagation</i>? Demostraremos a continuación la más importante de estas, la que permite actualizar los pesos de la red. Entender esto no sólo justifica la implementación de red neuronal que tenemos a la vista, nos lleva además a entender el cómo aprende una red neuronal artificial.  
<br> <img src=https://i.imgflip.com/1rks94.jpg?a469896
          width="100"/></br>

Supongamos una red neuronal de dos capas: una entrada de tamaño $i$ y una salida de tamaño $j$. Queremos minimizar el error de salida de la red, para lo cual pretendemos usar el método del gradiente. Esto quiere decir que queremos actualizar la salida de la red $\hat{y}$ en la dirección de la derivada del error: $$\frac{\partial E}{\partial \hat{y}}$$

Conocemos la función de pérdida $E$, así que conocemos su derivada respecto a $\hat{y}$. Pero no podemos manipular $\hat{y}$ directamente, sólo podemos hacer que cambie modificando los parámetros de la red, es decir los pesos $w$ que conectan cada neurona de la primera capa con cada neurona de la segunda capa, que conforman la matriz $W_{i \times j}$. 

$$ W = \begin{bmatrix}
  w_{11} & w_{12} & \dots & w_{1j} \\
  w_{21} & w_{22} & \dots & w_{2j} \\
  \vdots & \vdots & \ddots & \vdots \\
  w_{i1} & w_{i2} & \dots & w_{ij}
\end{bmatrix}$$

El método del gradiente indica que la actualización de la matriz de pesos que requerimos sería:
$$W \leftarrow W - \alpha \frac{\partial E}{\partial W}$$

<br> <img src=https://easyai.tech/wp-content/uploads/2019/01/tiduxiajiang-1.png
          width="400"/></br>

Donde la matriz gradiente $\frac{\partial E}{\partial W}$ es:

$$ \frac{\partial E}{\partial W} = \begin{bmatrix}
  \frac{\partial E}{\partial w_{11}} & \frac{\partial E}{\partial w_{12}} & \dots & \frac{\partial E}{\partial w_{1j}} \\
  \frac{\partial E}{\partial w_{21}} & \frac{\partial E}{\partial w_{22}} & \dots & \frac{\partial E}{\partial w_{2j}} \\
  \vdots & \vdots & \ddots & \vdots \\
  \frac{\partial E}{\partial w_{i1}} & \frac{\partial E}{\partial w_{i2}} & \dots & \frac{\partial E}{\partial w_{ij}}
\end{bmatrix}
$$ 

Por lo que la actualización a nivel de cada peso sería:
$$w_{ij} \leftarrow w_{ij} - \alpha \frac{\partial E}{\partial w_{ij}}$$

Pero, ¿cuánto vale $\frac{\partial E}{\partial w_{ij}}$? Usando la regla de la cadena (función multivariable) lo podemos expresar como:

$$ \frac{\partial E}{\partial w_{ij}} = \frac{\partial E}{\partial {\hat{y}_1}}\frac{\partial {\hat{y}_1}}{\partial w_{ij}}+\frac{\partial E}{\partial {\hat{y}_2}}\frac{\partial {\hat{y}_2}}{\partial w_{ij}}+\dots+\frac{\partial E}{\partial {\hat{y}_j}}\frac{\partial {\hat{y}_j}}{\partial w_{ij}} $$

Pero, ¿cuánto vale $\frac{\partial {\hat{y}_1}}{\partial w_{ij}}$? 

Observemos que $\hat{y}_{1}$ no depende de $w_{ij}$, así que esa componente de la derivada es cero. Lo mismo ocurre para todas las salidas $\hat{y}$, salvo $\hat{y}_j$ que sí depende de $w_{ij}$. 

Pero, ¿cuánto vale $\frac{\partial {\hat{y}_j}}{\partial w_{ij}}$?

Recordemos que:
$${\hat{y}}_{j} = x_{1}*w_{1j} + x_{2}*w_{2j} + \dots + x_{i}*w_{ij} + b $$

De lo que obtenemos que:
$$\frac{\partial {\hat{y}_j}}{\partial w_{ij}} = x_{i}$$

De lo que sale que:
$$ \frac{\partial E}{\partial w_{ij}} = \frac{\partial E}{\partial {\hat{y}_j}}x_{i} $$

Este resultado es válido para cualquier $i$ y cualquier $j$ en $w_{ij}$. Así que podemos volver a expresar la matriz gradiente de los pesos como: 

$$ \frac{\partial E}{\partial W} = \begin{bmatrix}
  \frac{\partial E}{\partial \hat{y}_{1}}x_{1} & \frac{\partial E}{\partial \hat{y}_{2}}x_{1} & \dots & \frac{\partial E}{\partial \hat{y}_{j}}x_{1} \\
  \frac{\partial E}{\partial \hat{y}_{1}}x_{2} & \frac{\partial E}{\partial \hat{y}_{2}}x_{2} & \dots & \frac{\partial E}{\partial \hat{y}_{j}}x_{2} \\
  \vdots & \vdots & \ddots & \vdots \\
  \frac{\partial E}{\partial \hat{y}_{1}}x_{i} & \frac{\partial E}{\partial \hat{y}_{2}}x_{i} & \dots & \frac{\partial E}{\partial \hat{y}_{j}}x_{i}
\end{bmatrix} = \begin{bmatrix}
  x_{1} & x_{2} & \dots & x_{i}
\end{bmatrix}
\cdot
\begin{bmatrix}
  \frac{\partial E}{\partial \hat{y}_{1}} \\
  \frac{\partial E}{\partial \hat{y}_{2}} \\
  \vdots \\
  \frac{\partial E}{\partial \hat{y}_{j}}
\end{bmatrix}
$$ 

De lo que resulta que:
$$ \frac{\partial E}{\partial W} = X^T \cdot \frac{\partial E}{\partial \hat{y}} $$

Acá conectamos con el punto inicial de este análisis, conocemos la función de pérdida $E$, así que conocemos su derivada respecto a $\hat{y}$. Por lo tanto, cuando ocurre backpropagation tenemos que actualizar los pesos de acuerdo a la siguiente fórmula:

$$W \leftarrow W - \alpha * X^T \cdot \frac{\partial E}{\partial \hat{y}}$$

Recordemos que $\alpha$ corresponde al <b><i>learning date</i></b>, que es la tasa con que castigamos el aprendizaje para fines facilitar convergencia más rápida pero también mitigando el riesgo de caer en óptimos locales o enfrentar no convergencia
<br> <img src=https://www.jeremyjordan.me/content/images/2018/02/Screen-Shot-2018-02-24-at-11.47.09-AM.png
          width="700"/></br>


El análisis anterior se ha realizado sobre el supuesto una red con una capa de entrada y una de salida. En redes más profundas se repite la misma mecánica durante el aprendizaje, capa por capa. De allí en nombre de esta fase: <b><i>backward propagation</i></b>. 
<br> <img src=https://miro.medium.com/v2/resize:fit:828/format:webp/1*0QPRST83oBicKPE_R4biJA.png
          width="500"/></br>


<b>Clase ActivationLayer</b>

In [ ]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

Esta capa aplica una función de activación a las salidas de la capa anterior. Necesita recibir como parámetros de iniciación la respectiva función de activación y la derivada de la función de activación. Los pesos (incluida la constante se sesgo) se inician en valores aleatorios. En el método forward_propagation, simplemente aplica la función de activación a la entrada. En el método backward_propagation, se calcula el error de entrada como el error de salida multiplicado por la derivada de la función de activación.

<b>Clase Network</b>

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        input_data = np.array([[x] for x in input_data])
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        x_train = np.array([[x] for x in x_train])
        samples = len(x_train)
        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # recurrimos al método Predict que hace fordward propagation
                output = self.predict(x_train[j])[0] # La respuesta es un array, desempacamos.

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            
            # Para usar en clasificación (con más de dos clases)
            # calculamos el error promedio entre nodos de salida.
            err = np.mean(err)
            
            # Imprimomos el error promedio de cada época, más que nada
            # para seguimiento del aprendizaje. 
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

Esta es la red neuronal en sí. Contiene una lista de capas y la función de pérdida a utilizar, así como la derivada en $y$ de la misma.
    
<ul>
    <li>El método <b>add</b> se utiliza para añadir capas a la red.</li>
    <li>El método <b>use</b> se utiliza para establecer la función de pérdida.</li>
    <li>El método <b>predict</b> se utiliza para calcular las salidas de la red para un conjunto dado de entradas.</li>
    <li>El método <b>fit</b> se utiliza para entrenar la red con un conjunto de datos de entrenamiento. Durante cada época, para cada muestra, primero se realiza la propagación hacia adelante para calcular las salidas, y luego se realiza la propagación hacia atrás para actualizar los pesos.</li>
</ul>









Si ponemos atención al método fit veremos que hace uso sucesivamente de forward propagation y backward propagation, en la primera usa la función de pérdida para computar el error, mientras que en la segunda usa la derivada de la función de pérdida para transmitir la señal que tiene que actualizar hacia atrás los pesos de la red. Este es el punto donde estamos alimentando la pieza que hace posible computar la derivada del error respecto a los pesos de cada capa, permitiendo así que se actualicen, desde la capa de salida hacia la capa de entrada de la red.

Pero, por ahora, aún no hemos visto de donde sale la función de pérdida y la derivada de la función de pérdida, sólo está asumiendo que están disponibles como parámetros de entrada a la red. 

Además, podemos observar que se va a imprimir el error (pérdida) computado en cada epoch. 

<b>Funciones de Activación y Pérdida </b>

Las clases previas permiten construir la Red Neuronal, pero para que esta pueda operar necesita definir previamente funciones que operan como parámetros, que corresponden a dos categorías: funciones de activación y función de pérdida. Cada una de estas va a necesitas sus correspondientes derivadas durante el proceso de back propagation. 

La capa de activación depende de una <b>función de activación y su derivada en $x$</b>, por lo que necesitamos establecer dicha función. En las capas intermedias podemos usar Tangente Hiperbólica como función de activación, la cual tiende a retornar valores -1 ó 1. 

$$\tanh(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}$$

<br> <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Hyperbolic_Tangent.svg/1200px-Hyperbolic_Tangent.svg.png
          width="350"/>

Es necesario también proveer de la derivada de la función tangente hiperbólica:
          
$$\frac{{d}}{{dx}}\tanh(x) = 1 - \tanh^2(x)$$

En capa de salida podríamos requerir otra función de activación, por ejemplo en el caso en que tenemos clasificación entre valores 0 y 1. La función sigmoidal sería la opción por efecto. 

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

<br> <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1200px-Logistic-curve.svg.png
          width="350"/>
          
La derivada de la función sigmoidal es:

$$ \frac{{d}}{{dx}}\sigma(x) = \sigma(x) \cdot (1 - \sigma(x)) $$




In [ ]:
import numpy as np

# activation function and its derivative
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

Al igual que con la función de activación, también vamos a necesitar especificar la <b>función de pérdida (error)</b>: 
$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

También hace falta la <b>derivada de la función de pérdida en $y$</b>:
$$\frac{{\partial \text{MSE}}}{{\partial \hat{y}_i}} = \frac{2}{n} \sum_{i=1}^{n} (\hat{y}_i - y_i)$$

Sin embargo, a menos que implementemos <i>minibath</i>, el error se calculará para cada observación, así que no necesitamos considerar $y$ e $\hat{y}$ como vectores. 

In [ ]:
import numpy as np

# loss function and its derivative
def mse(y_real, y_hat):
    return (y_real-y_hat)**2

def mse_prime(y_real, y_hat):
    return 2*(y_hat-y_real)

A continuación, creamos un modelo de clasificación usando la clase Network, le agregamos capas intermedias declarando en cada una su tamaño de entrada y salida. Cada capa full conected debe ir acompañada de una capa de activación. La capa de entrada necesita corresponder a la forma de $X$, mientras que la salida corresponde a la forma de $y$. Hemos definido un parámetros de epochs para controlar la cantidad de iteraciones permitidas a la red. 

<b>Datos para probar</b>

Cuando parecía que ya se había terminado de hundir, vamos a reflotar al Titanic, una vez más. Este set de datos ya lo conocemos, puesto que lo usamos muchas veces como ejemplo en la asignatura Ciencia de Datos. 


In [ ]:
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

# Una vez más cargamos y procesamos el Titanic
file = open("Titanic.csv", "r")
titanic = file.read()
file.close()
titanic = titanic.split("\n")
for j in range(len(titanic)):
    titanic[j] = titanic[j].split(",")

# El nombre tiene ',' así que necesitamos volver a unirlo.
for elemento in titanic[1:]:
    elemento[3] = elemento[3]+elemento[4]
    del elemento[4]
titanic = np.array(titanic)
titanic = np.delete(titanic, [3,8,10,11], axis=1)

# Codificamos género como 0-1
for linea in titanic[1:]:
    linea[3] = '0' if linea[3] == 'male' else '1'

# Quitamos las observaciones con datos perdidos
borrar = []
for l in range(len(titanic)):
    if '' in titanic[l]:
        borrar.append(l)
titanic = np.delete(titanic, borrar, axis=0)

# Arreglamos vector y matriz como array
y = titanic[:,1][1:]
X = titanic[:,2:][1:]
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

# Aplicamos Normalización Min-Max
X= MinMaxScaler().fit_transform(X)

# Separamos la muestra al azar, 80% para entrenar, 20% para testeo final. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

Usemos la implementación que ya tenemos, creando una instancia de red neuronal invocando la clase network, y a continuación agregamos dos capas intermedias. Cada una de estas capas intermedias tiene, a su vez, dos capas: una full conected y otra de activación. En cada capa full conected debemos indicar el tamaño de entrada (número de nodos de la capa anterior) y salida (número de nodos de la capa). La capa full conected inicial requiere como tamaño de entrada la cifra de dimensiones con que se presenta el problema. 

En cada capa de activación debemos entregar la función de activación y la derivada de la función de activación. Cerramos la red con una capa de salida, cuyo tamaño de salida debe corresponder al tamaño de la solución esperada. 
En este caso, probemos con una red con dos capas ocultas de 10 nodos con activación tangente hiperbólica, y con función sigmoidal en la capa de salida.  

In [ ]:
# Crear instancia de Network
model = Network()

# y agregamos capas al modelo. Debemos usarcapas en pares: (FC, Activartion)
# las capas FC es donde declaramos la cantidad de nodos. Debemos hacer calzar entradas con salidas.
# las capas de activación tienen las funciones de activación (y derivada) que decidamos usar.
model.add(FCLayer(6, 10))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(10, 10))                                   
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(10, 1))                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

Para entrenar la red falta establecer la función de pérdida que define el error, y la derivada de la misma función que se requiere durante back propagation. 

El entrenamiento requiere proveer los datos de entrenamiento, dividido en la matriz $X$ con variables predictoras y el vector $y$ de resultados. Además, se necesita establecer dos parámetros: la tasa de aprendizaje (learning rate) y número de iteraciones (epochs). Probemos nuestra red con $10$ epochs y $0.1$ de learning rate. 


In [ ]:
# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

La evaluación de los resultados obtenidos por el modelo la debemos hacer sobre el set de datos de testeo que hemos reservado para estos efectos. Podemos observar la calidad de los resultados computando el accuracy y matriz de confusión.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Nuestra red devuelve arrays en la respuesta, debemos desempacarlo.
# Además, la respuesta es una probabilidad (entre 0 y 1), lo necesitamos como 0 o 1. 
y_hat = [round(y[0][0],0) for y in y_hat]

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

<b>Reconocimiento de Imágenes con MNIST</b>

Ya nos hemos aburrido del Titanic, al final siempre se hunde. 

Vamos a usar un dataset disponible en Keras, de forma que no tendremos que tener nuestros propios datos. Se trata de <b><i>Mnist</i></b>, una colección de imágenes de dígitos (números) escritos a mano por distintas personas, con la que ya trabajamos durante Ciencia de Datos. 
<br> <img src=https://www.ttested.com/gallery/thumbnails/ditch-mnist.jpg
          width="400"/>

In [ ]:
import numpy as np  
from keras.datasets import mnist
import random
import matplotlib.pyplot as plt
import tabulate

random.seed(1234)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X, y = zip(*random.sample(list(zip(X_train, y_train)), 2))

print(np.array(X).shape)

print(tabulate.tabulate(X[0]))
fig = plt.figure
plt.imshow(X[0], cmap='gray_r')
plt.show()
print(y[0])
print()
print()

print(tabulate.tabulate(X[1]))
fig = plt.figure
plt.imshow(X[1], cmap='gray_r')
plt.show()
print(y[1])

In [ ]:
# No necesitamos tantos datos.
(X_train, y_train), (X_test, y_test) = mnist.load_data()
random.seed(123) # Vamos a controlar la aleatoriedad en adelante. 
X, y = zip(*random.sample(list(zip(X_train, y_train)), 2000))

# Sí necesitamos que la forma de X sea la de un vector, en lugar de una matriz. 
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')
X = np.reshape(X, (X.shape[0], -1))

# Normalizamos Min-Max
X= MinMaxScaler().fit_transform(X)

# Dividomos la muestra en dos, una para entrenar y otra para testing, como tenemos 
# muestra de sobra nos damos el lujo de testear con la misma cantidad que entrenamos.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

# Necesitamos que y_train sea un valor categórico, en lugar de un dígito entero.
from keras.utils import to_categorical
print('y_train antes de categorizar: ',y_train[0])
y_train = to_categorical(y_train)
print('y_train después de categorizar: ',y_train[0])

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende de la cantidad de Xs.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 16))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(16, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

In [ ]:
# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Ya obtenemos resultados bastante buenos. Incrementemos la capacidad de representación del modelo, por la vía de aumentar la cantidad de capas, es decir haremos la red más profunda. 

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende de la cantidad de Xs.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 16))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(16, 16))                           
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(16, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Profundizar la red no necesariamente mejora el aprendizaje, especialmente al momento de generalizar sobre el set de datos de prueba. 

Probemos incrementar la cantidad de neuronas de la red con una capa oculta.  

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende de la cantidad de Xs.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 32))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(32, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Aparentemente más neuronas en una capa funciona mejor. Hagamos la prueba de incrementar más fuertemente el número de neuronas. 

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende de la cantidad de Xs.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 200))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(200, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Incrementar el número de neuronas también termina deteriorando la capacidad de aprender del modelo. 

Dejemos la red como estaba al principio, pero entrenémosla más. 

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende de la cantidad de Xs.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 16))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(16, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=50, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Es probable que al aumentar la capacidad de representación de la red, ya sea por disponer de más capas, más neuronas o más tiempo de entrenamiento, estamos generando las condiciones que facilitan el <b><i>overfitting</i></b>. Hasta ahora hemos tratado de controlarlo por la vía de seleccionar los parámetros de configuración del modelo buscando que no tenga demasiada capacidad. Sin embargo, con esto estamos limitando nuestra posibilidad de aprovechar modelos más robustos. El aprendizaje profundo, que ha permitido generar los modelos AI que están cambiando el mundo, requiere que podamos enfrentar este problema por otras vías.  


<b> CONCLUSIÓN </b>

Este apunte profundiza en la mecánica del aprendizaje de una red neuronal, que podemos resumir en una serie de operaciones de algebra matricial. Usando solo recursos genéricos de Python hemos demostrado que se puede construir redes neuronales con la capacidad de aprender sobre distintos problemas. Si bien hemos usado una implementación previamente publicada, hemos procurado explicar la matemática que justifica dicha implementación, demostrando su validez teórica. Luego hemos realizado experimentos donde se aprecia la capacidad de aprendizaje y generalización de los modelos construidos, demostrando empíricamente la validez de estos. 

Buena parte de la asignatura consistirá en extender esta implementación, incorporando distintos métodos para mejorar el aprendizaje de nuestra red neuronal, y adaptándola a distintos contexto de problema. Esto lo hacemos con fines pedagógicos, puesto que en general lograríamos los mismos y mejores resultados usando librerías que ya están disponibles. También habrá capítulos de la asignatura donde nos apoyaremos en esas implementaciones que ya existen, cuando abordemos problemas crecientemente complejos. Pero siempre será satisfactorio saber que podemos implementar nuestra propia red neuronal. 

<img src=https://img2-levelup.buscafs.com/415775_945x532.jpg width="200"/>
<center>CON JUEGOS DE AZAR Y PERSONUELAS</center>